In [2]:
%pip install --upgrade pip
%pip install --upgrade datasets[audio] accelerate transformers accelerate evaluate jiwer tensorboard gradio
%pip install evaluate


Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: datasets[audio]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset, Audio

# Load all JSONL files in the transcripts_folder
data_files = "transcripts_folder/*.jsonl"
dataset = load_dataset("json", data_files=data_files)

# Optionally, split into train and validation sets if needed
split_datasets = dataset["train"].train_test_split(test_size=0.1)
ds_train = split_datasets["train"]
ds_eval = split_datasets["test"]


/opt/anaconda3/envs/audio_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from transformers import (WhisperForConditionalGeneration, WhisperProcessor, 
                          WhisperTokenizer, WhisperFeatureExtractor, 
                          Seq2SeqTrainer, Seq2SeqTrainingArguments)
model_name_or_path = "openai/whisper-small"

In [9]:
import torch
from datasets import load_dataset, Audio
import evaluate
from dataclasses import dataclass

# 2. Define model and processor parameters.
model_name_or_path = "openai/whisper-small"
source_language = "english"   # Audio is in English
target_language = "hindi"     # We want translation into Hindi
task = "translate"            # Translation task

# Load processor and model.
whisper_processor = WhisperProcessor.from_pretrained(model_name_or_path, language=source_language, task=task)
whisper_model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
whisper_model.to(device)

# Load feature extractor and tokenizer.
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=source_language, task=task)

# Override the decoder start token to Hindi.
forced_decoder_ids = whisper_processor.get_decoder_prompt_ids(language="hindi", task="translate")

# 3. Prepare the dataset: Use the "translation" field for target Hindi text.
def prepare_dataset(batch):
    audio = batch["audio"]
    # Compute log-Mel features from the audio array.
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=16000).input_features[0]
    # Tokenize the Hindi translation.
    batch["labels"] = tokenizer(batch["translation"],truncation=True,
        max_length=1024).input_ids
    return batch

ds_train = ds_train.map(prepare_dataset, num_proc=4)
ds_eval = ds_eval.map(prepare_dataset, num_proc=4)

# 4. Define a data collator to handle padding.
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: any
    decoder_start_token_id: int

    def __call__(self, features):
        # Pad input features.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        # Pad label sequences.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        
        # Optionally remove the decoder start token if it is pre-pended.
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=whisper_processor,
    decoder_start_token_id=forced_decoder_ids,
)

# 5. Define BLEU metric and compute_metrics function.
metric_bleu = evaluate.load("bleu")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with the pad token id.
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # Decode predictions and labels.
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Tokenize for BLEU.
    predictions = [p.split() for p in pred_str]
    references = [[ref.split()] for ref in label_str]

    bleu = metric_bleu.compute(predictions=predictions, references=references)
    return {"bleu": bleu["bleu"] * 100}  # Percentage score

# 6. Set up training arguments.
total_steps = len(ds_train) // 16  # Adjust based on batch size.
training_args = Seq2SeqTrainingArguments(
    output_dir="whisper_finetuned_en_hi",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=int(0.1 * total_steps),
    num_train_epochs=50,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
)

# 7. Initialize the Seq2SeqTrainer.
trainer = Seq2SeqTrainer(
    model=whisper_model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=whisper_processor.feature_extractor,
)

# 8. Fine-tune the model.
trainer.train()

# 9. Save the final (or best) model.
trainer.save_model("whisper_finetuned_en_hi_final")

Map (num_proc=4): 100%|██████████| 7/7 [00:00<00:00, 21.75 examples/s]


ValueError: fp16 mixed precision requires a GPU (not 'mps').